In [5]:
#Import
from decision_tree import DecisionTree as DT
from decision_tree_for_forest import DecisionTree as DT2
from random_forest import RandomForest
from sklearn.model_selection import KFold
import numpy as np 
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import itertools

In [6]:
#Data
data=pd.read_csv('coffee_data.csv')
# data=pd.read_csv('wine_dataset_small.csv')
np_array=data.to_numpy()
X, y = np_array[:, :-1], np_array[:, -1]

In [7]:
rf = RandomForest(
    n_estimators=20, 
    max_depth=7, 
    criterion="gini", 
    max_features="log2"
)

In [8]:
k = 10
# Create StratifiedKFold instance (to keep class distribution in all folds)
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

# To store accuracy for each fold
accuracies = []

# Perform k-fold cross-validation
for train_index, val_index in skf.split(X, y):
    X_train_fold, X_val_fold = X[train_index], X[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    # Fit the RandomForest on the current fold's training data
    rf.fit(X_train_fold, y_train_fold)
    
    # Make predictions on the validation set
    y_val_pred = rf.predict(X_val_fold)
    
    # Calculate accuracy for this fold
    fold_accuracy = accuracy_score(y_val_fold, y_val_pred)
    
    # Append accuracy to list
    accuracies.append(fold_accuracy)

# Calculate the average accuracy across all folds
avg_accuracy = np.mean(accuracies)

print(f"K-Fold Cross-Validation Accuracies: {accuracies}")
print(f"Average Cross-Validation Accuracy: {avg_accuracy}")

K-Fold Cross-Validation Accuracies: [0.7380952380952381, 0.8809523809523809, 0.8095238095238095, 0.8571428571428571, 0.7857142857142857, 0.8333333333333334, 0.7619047619047619, 0.8333333333333334, 0.8571428571428571, 0.7560975609756098]
Average Cross-Validation Accuracy: 0.8113240418118467


In [9]:
# decision_tree_params = {
#     'max_depth': [None, 3, 5, 7, 10, 12, 15, 17, 20],
#     'criterion': ['gini', 'entropy'],
#     'max_features': [None, 'sqrt', 'log2', 'two']
# }

decision_tree_params = {
    'max_depth': [None, 1, 2, 3, 4],
    'criterion': ['gini', 'entropy'],
    'max_features': [None, 'sqrt', 'log2', 'two', 'minus_one']
}

best_accuracy = 0
best_params = None

for max_depth, criterion, max_features in itertools.product(
    decision_tree_params['max_depth'],
    decision_tree_params['criterion'],
    decision_tree_params['max_features']
):
    accuracies = []
    for train_index, val_index in skf.split(X, y):
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]
        
        # Initialize and train the model
        dt = DT2(
            max_depth=max_depth,
            criterion=criterion,
            max_features=max_features
        )
        dt.fit(X_train_fold, y_train_fold)
        
        # Evaluate on validation fold
        y_pred = dt.predict(X_val_fold)
        accuracy = np.mean(y_pred == y_val_fold)
        accuracies.append(accuracy)
    
    avg_accuracy = np.mean(accuracies)
    
    if avg_accuracy > best_accuracy:
        best_accuracy = avg_accuracy
        best_params = {
            'max_depth': max_depth,
            'criterion': criterion,
            'max_features': max_features
        }

print("Best Decision Tree parameters:", best_params)
print("Cross-Validation Accuracy:", best_accuracy)

TypeError: DecisionTree.__init__() got an unexpected keyword argument 'max_features'